<a href="https://colab.research.google.com/github/jsscmaina/machinelearning4102/blob/master/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from numpy import array
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV

In [28]:
from google.colab import files
upload = files.upload()

Saving train_data.csv to train_data (1).csv


In [0]:
data = pd.read_csv('train_data.csv')

In [134]:
#droping missing values
#data.dropna(subset=['Alley','FireplaceQu'],inplace=True)
#data.dropna(axis=1,inplace=True)

#replacing missing values with zero so that numeric data still have float or int datatype
data.fillna(0, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          1460 non-null   object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [135]:
nominal=['Street','Alley','Utilities','LotConfig','Neighborhood','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','Foundation',
         'Heating','Electrical','GarageType','GarageFinish','PavedDrive','PoolQC','Fence','MiscFeature','SaleType','SaleCondition','ExterQual',
         'ExterCond','BsmtCond','KitchenQual','FireplaceQu','GarageQual','GarageCond']
ordinal=['MSZoning','LotShape','LandContour','LandSlope','Condition1','Condition2','BldgType','HouseStyle','BsmtQual','BsmtExposure',
         'BsmtFinType1','BsmtFinType2','HeatingQC','CentralAir','Functional']
target=['SalePrice']
numerical=['LotFrontage','LotArea','OverallQual','OverallCond','YearRemodAdd','YearBuilt','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF',
           'TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr',
           'TotRmsAbvGrd','Fireplaces','GarageYrBlt','GarageCars','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch',
           'PoolArea','MiscVal','MoSold','YrSold']

categorical_data = data.select_dtypes(include=['object']).copy()
column_names = list(categorical_data.columns)
#categorical_data
column_names
for feature in column_names:
  categorical_data[feature]=categorical_data[feature].astype('category')
for feature in ordinal:
      categorical_data[feature]=categorical_data[feature].cat.codes
categorical_data.head(20)

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,3,Pave,0,3,3,AllPub,Inside,0,CollgCr,2,2,0,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,3,TA,4,3,6,GasA,0,1,SBrkr,Gd,6,0,Attchd,RFn,TA,TA,Y,0,0,0,WD,Normal
1,3,Pave,0,3,3,AllPub,FR2,0,Veenker,1,2,0,2,Gable,CompShg,MetalSd,MetalSd,None,TA,TA,CBlock,3,TA,2,1,6,GasA,0,1,SBrkr,TA,6,TA,Attchd,RFn,TA,TA,Y,0,0,0,WD,Normal
2,3,Pave,0,0,3,AllPub,Inside,0,CollgCr,2,2,0,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,3,TA,3,3,6,GasA,0,1,SBrkr,Gd,6,TA,Attchd,RFn,TA,TA,Y,0,0,0,WD,Normal
3,3,Pave,0,0,3,AllPub,Corner,0,Crawfor,2,2,0,5,Gable,CompShg,Wd Sdng,Wd Shng,None,TA,TA,BrkTil,4,Gd,4,1,6,GasA,2,1,SBrkr,Gd,6,Gd,Detchd,Unf,TA,TA,Y,0,0,0,WD,Abnorml
4,3,Pave,0,0,3,AllPub,FR2,0,NoRidge,2,2,0,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,3,TA,1,3,6,GasA,0,1,SBrkr,Gd,6,TA,Attchd,RFn,TA,TA,Y,0,0,0,WD,Normal
5,3,Pave,0,0,3,AllPub,Inside,0,Mitchel,2,2,0,0,Gable,CompShg,VinylSd,VinylSd,None,TA,TA,Wood,3,TA,4,3,6,GasA,0,1,SBrkr,TA,6,0,Attchd,Unf,TA,TA,Y,0,MnPrv,Shed,WD,Normal
6,3,Pave,0,3,3,AllPub,Inside,0,Somerst,2,2,0,2,Gable,CompShg,VinylSd,VinylSd,Stone,Gd,TA,PConc,1,TA,1,3,6,GasA,0,1,SBrkr,Gd,6,Gd,Attchd,RFn,TA,TA,Y,0,0,0,WD,Normal
7,3,Pave,0,0,3,AllPub,Corner,0,NWAmes,4,2,0,5,Gable,CompShg,HdBoard,HdBoard,Stone,TA,TA,CBlock,3,TA,3,1,2,GasA,0,1,SBrkr,TA,6,TA,Attchd,RFn,TA,TA,Y,0,0,Shed,WD,Normal
8,4,Pave,0,3,3,AllPub,Inside,0,OldTown,0,2,0,0,Gable,CompShg,BrkFace,Wd Shng,None,TA,TA,BrkTil,4,TA,4,6,6,GasA,2,1,FuseF,TA,2,TA,Detchd,Unf,Fa,TA,Y,0,0,0,WD,Abnorml
9,3,Pave,0,3,3,AllPub,Corner,0,BrkSide,0,0,1,1,Gable,CompShg,MetalSd,MetalSd,None,TA,TA,BrkTil,4,TA,4,3,6,GasA,0,1,SBrkr,TA,6,TA,Attchd,RFn,Gd,TA,Y,0,0,0,WD,Normal


In [136]:
data_copy = categorical_data.copy()

data_copy = pd.concat([pd.get_dummies(data_copy[nominal]),data_copy],axis=1)
data_copy.drop(nominal,axis=1,inplace=True)
#data_copy
new_data = pd.concat([data_copy,data[numerical],data[target]],axis=1)
new_data

,Street_Grvl,Street_Pave,Alley_0,Alley_Grvl,Alley_Pave,Utilities_AllPub,Utilities_NoSeWa,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,...,BsmtFinType2,HeatingQC,CentralAir,Functional,LotFrontage,LotArea,OverallQual,OverallCond,YearRemodAdd,YearBuilt,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,6,0,1,6,65.0,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2,2008,208500
1,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,6,0,1,6,80.0,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,5,2007,181500
2,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,6,0,1,6,68.0,11250,7,5,2002,2001,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,9,2008,223500
3,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,6,2,1,6,60.0,9550,7,5,1970,1915,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2,2006,140000
4,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,...,6,0,1,6,84.0,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,6,0,1,6,62.0,7917,6,5,2000,1999,0.0,0,0,953,953,953,694,0,1647,0,0,2,1,3,1,7,1,1999.0,2,460,0,40,0,0,0,0,0,8,2007,175000
1456,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,...,5,4,1,2,85.0,13175,6,6,1988,1978,119.0,790,163,589,1542,2073,0,0,2073,1,0,2,0,3,1,7,2,1978.0,2,500,349,0,0,0,0,0,0,2,2010,210000
1457,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,6,0,1,6,66.0,9042,7,9,2006,1941,0.0,275,0,877,1152,1188,1152,0,2340,0,0,2,0,4,1,9,2,1941.0,1,252,0,60,0,0,0,0,2500,5,2010,266500
1458,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,5,2,1,6,68.0,9717,5,6,1996,1950,0.0,49,1029,0,1078,1078,0,0,1078,1,0,1,0,2,1,5,0,1950.0,1,240,366,0,112,0,0,0,0,4,2010,142125


In [137]:
all_features = list(new_data.columns)
all_features
X = new_data.loc[:,all_features].values
Y = new_data.loc[:,["SalePrice"]].values
X

array([[0.00000e+00, 1.00000e+00, 1.00000e+00, ..., 2.00000e+00,
        2.00800e+03, 2.08500e+05],
       [0.00000e+00, 1.00000e+00, 1.00000e+00, ..., 5.00000e+00,
        2.00700e+03, 1.81500e+05],
       [0.00000e+00, 1.00000e+00, 1.00000e+00, ..., 9.00000e+00,
        2.00800e+03, 2.23500e+05],
       ...,
       [0.00000e+00, 1.00000e+00, 1.00000e+00, ..., 5.00000e+00,
        2.01000e+03, 2.66500e+05],
       [0.00000e+00, 1.00000e+00, 1.00000e+00, ..., 4.00000e+00,
        2.01000e+03, 1.42125e+05],
       [0.00000e+00, 1.00000e+00, 1.00000e+00, ..., 6.00000e+00,
        2.00800e+03, 1.47500e+05]])

In [138]:
X = StandardScaler().fit_transform(X)
X

array([[-0.06423821,  0.06423821,  0.25782141, ..., -1.5991111 ,
         0.13877749,  0.34727322],
       [-0.06423821,  0.06423821,  0.25782141, ..., -0.48911005,
        -0.61443862,  0.00728832],
       [-0.06423821,  0.06423821,  0.25782141, ...,  0.99089135,
         0.13877749,  0.53615372],
       ...,
       [-0.06423821,  0.06423821,  0.25782141, ..., -0.48911005,
         1.64520971,  1.07761115],
       [-0.06423821,  0.06423821,  0.25782141, ..., -0.8591104 ,
         1.64520971, -0.48852299],
       [-0.06423821,  0.06423821,  0.25782141, ..., -0.1191097 ,
         0.13877749, -0.42084081]])

In [142]:
pca = PCA(n_components=2)
pc_s = pca.fit_transform(X)
pca_data = pd.DataFrame(pc_s,columns=['PC1','PC2'])
#pca_data
final_data = pd.concat([pca_data,data[target]],axis=1)
final_data

,PC1,PC2,SalePrice
0,3.453933,0.781387,208500
1,-0.135608,-2.690263,181500
2,4.148528,0.431035,223500
3,-1.428240,-0.129921,140000
4,5.960017,0.453432,250000
...,...,...,...
1455,1.718408,0.423209,175000
1456,0.634842,-3.801212,210000
1457,1.285735,0.395389,266500
1458,-2.951654,-2.449362,142125


In [143]:
pca.explained_variance_ratio_

array([0.07274969, 0.03288561])